# Creating a sales dashboard in Tableau Desktop

## Goals

In this case, you will learn how to plan and build a visual dashboard in Tableau Desktop. You will expand upon the skills you learned about choosing visualizations and presenting them in a clear and compelling way.

## Introduction

**Business Context.** You are a member of the business analytics team of *Simply Music*, a Florida-based musical instrument store that has been growing at a fast pace over the last few years. Management is keen to find out what the most popular items are and which locations are the most profitable, in order to adjust their marketing strategies and their stock logistics to increase sales and revenue. They also want to learn about the number of deliveries made outside Florida, since they are contemplating opening a new branch in another state.

**Business Problem.** You are tasked with creating a sales dashboard for management to use. This dashboard must display sales, revenue, and profit over time, and allow for queries across product, location, and customer age. From your previous meetings with the management, you know that they are interested in having these three broad questions answered:

1. How have our sales, revenue and profits evolved over time?
2. What are the locations with the highest numbers of sales and profit?
3. What are the items with the highest numbers of sales, profit and revenue across: product description, product type, product line, location, customer age group and state of delivery?

**Analytical Context.** We have been given a sample dataset of 5000 sales records from Simply Music, corresponding to the years 2018 and 2019. They ask you to create the dashboard with this sample data, and after this is done and your dashboard receives the management's approval, your team will have to connect it to an online database to feed it the real time data that comes from the company's sales tracking software. The sample table and the database share the same structure, so you don't have to worry about database management or creation. You will plan your dashboard and deploy it using [Tableau Desktop](https://www.tableau.com/products/desktop).

The case is structured as follows: you will 1) understand the business process that the dashboard has to represent; 2) plan the dashboard layout and visualizations; and 3) create your dashboard using Tableau.

In [ ]:
# We will use pandas to explore our data before uploading it to Tableau (we won't modify the data)
import pandas as pd

## Understanding the business process

Data professionals are often required to extract valuable information from datasets without being given specific questions to answer. We usually address these cases with exploratory data analysis combined with domain knowledge. Other times, we can get the clients to articulate a limited set of questions. There is a third scenario, where the clients want to explore the data themselves in such a way that they can easily answer *ad-hoc* queries that can't possibly be known in advance. This is much more open-ended and requires us to intimately understand the natural structure of the data and likely queries.

One way to address this need is to create dashboards that organize and display all relevant business information, as well as various filters that users might want to have, while being fully extensible if required in the future. When you create dashboards, you define a set of relevant variables and feed them into interactive visualizations that the users can then filter at will to explore relationships between the variables.

Dashboards work best when they mirror the business processes that underlie the data. It is always advised that you study the greater business context so that you can put yourself into the shoes of your end users (in this case, management) and deliver a product that is user-friendly and pertinent.

### Specifics of <i>Simply Music</i>

Simply Music is a retailer that sells mostly to individual buyers who buy in small quantities. The customers are not restricted to a single state, and in fact many of them buy online from all over the country. But they also have three physical shops located in the state of Florida. These are the product lines that the store serves:

1. Musical instruments
    1. Acoustic pianos
    2. Digital pianos and keyboards
    3. Guitars (electric and acoustic)
    4. Woodwinds (flutes, oboes, bassoons, saxophones, etc.)
    5. Brass (trumpets, horns)
    6. Synthetics
    7. Strings (everything bowed: violins, cellos, etc.)
    8. Percussion (mostly drum kits)
2. Accessories
    1. Strings (for guitars, violins, etc.)
    2. Pedals and amps (for electric guitars)
    3. Sheet music

Furthermore, these are the four selling channels that the company uses:

1. Physical store in Fort Lauderdale
2. Physical store in Tampa
3. Physical store in West Palm Beach
4. Online store

Each transaction is recorded at the time of purchasing, whether the purchase happens online or at a physical location. Accepted means of payment include cash and credit card. If the customer is not happy with their product, they can file a complaint within 10 days of the purchase.

## Looking at the data

The database team has given us access to a sample of 5,000 transactions and some other relevant data.

The first table is called `transactions`. Let's look at it using `pandas` and the `.head()` method:

In [ ]:
transactions = pd.read_csv('data/transactions.csv')
transactions.head()

There are several attributes in this table, namely:

1. **TIME_KEY**: A key that refers to the year, month and day of the purchase, in YYMMDD format (e.g. `180122` means January 22, 2018)
2. **PRODUCT_KEY**: A number that uniquely identifies the product
3. **UNITS_SOLD**: The number of units sold
4. **DOLLARS_SOLD**: The total amount of dollars sold of this product. This variable is equal to the units sold times the unit price
5. **CHANNEL**: Either `Fort Lauderdale`, `Tampa`, `West Palm Beach` or `Website`
6. **CUSTOMER_KEY**: A number that uniquely identifies the customer
7. **DESTINATION_STATE**: The state to which the customer wants their products delivered. Available only for online transactions

Additionally, we also have the `products` table:

In [ ]:
products = pd.read_csv('data/products.csv')
products.head()

The attributes are:

1. **PRODUCT_KEY**: The unique identifier of the product
2. **PRODUCT_LINE**: The product line (either "Musical instruments" or "Accessories")
3. **PRODUCT_TYPE**: The type of the product (e.g. "Acoustic pianos", "Brass")
4. **DESCRIPTION**: The name of the product (e.g. "Casio Digital Piano AP650")
5. **COST**: How much the company estimates the item cost them (includes fixed and variable costs) - this is also known as the unit cost
6. **UNIT_PRICE**: The unit price

Let's also load in the `time` and `customers` tables:

In [ ]:
time = pd.read_csv('data/time.csv')
time.head()

The attributes of the `time` table are:

1. **TIME_KEY**: The time key in YYYYMMDD format (e.g. `20180122` means January 22, 2018)
2. **YEAR**: The year
3. **MONTH**: The month in MM format (e.g. February means `02`)
4. **DAY**: The day in DD format
5. **WEEK_OF_YEAR**: The week of the year (the week of January 1st counts as week 1)
6. **QUARTER**: The quarter to which the month belongs

Notice that we don't have data for hours or minutes.

In [ ]:
customers = pd.read_csv('data/customers.csv')
customers.head()

The fields of the `customers` table are:

1. **CUSTOMER_KEY**: The unique identifier of the customer (can be a social security number or any other number)
2. **CUSTOMER_NAME**: The customer's full name
3. **CUSTOMER_AGE**: The age of your customer
4. **CUSTOMER_AGE_GROUP**: The customer's age group

You might be wondering how we're going to pool all this data in order to create our dashboard. The good news is that Tableau offers a really easy way to merge tables, so in many scenarios you don't have to worry too much about that. The other good news is that in this particular case, we've already pre-merged the data for you in the `simply_music.csv` dataset (which you can explore using `pd.read_csv('data/simply_music.csv)` if you like).

## Choosing the right visualizations

Let's keep in mind our problem statement: "The dashboard must display sales, revenue and profit across product type, location and customer age." What visualizations are most appropriate given this?

Tableau offers a variety of visualizations, ranging from tables and plots to simple geographical maps. We'll review several options to help us decide which ones suit our needs best.

You might recall that data can be *categorical* or *numerical*. In turn, numerical data can be of type *integer* or *float*. Data types are probably the key criterion to consider when it comes to choosing a visualization. Secondary criteria include ease of interpretation and the familiarity of your end user with a specific kind of plot.

### Categorical vs. numerical

When you need to plot a categorical variable against a numerical variable, your most likely choices are bar charts and pie charts, which we have already discussed in previous cases. A third option is using **gauges**. If you have variables whose values happen to lie inside the same given range, then you can plot them as gauge meters:

<img src="data/img/gauge.jpg" width="500"/>

Finally, if your categorical variable is geographical (states, countries), you can create **choropleth maps** (the numerical variable is represented as a hue):

<img src="data/img/choropleth.jpg" width="500"/>

A useful variant is using bubbles instead of colors, where the size of the bubble (not the color) represents the magnitude of the value.

### Numerical vs. numerical

Here, you can't go wrong with scatterplots. You can even add a third numerical variable and a categorical variable into the mix, and create a **bubble chart**, in which the third numerical variable is represented as the size of the bubble and the categorical variable maps to a color code.

### Numerical over time

Here you might find line plots to be handy. You can add more series to the plot (i.e. more lines) to compare across different numerical variables over time. When we add more series, we usually differentiate them using a different color for each one. Here's an example:

<img src="data/img/line_chart.jpg" width="500"/>

### Tables and scorecards

The last visualizations we'll cover aren't actually plots. You can always represent your data in tabular form, and that's sometimes more appropriate than a visual chart. The general principle is that you should keep tables simple and uncluttered. If that's not possible, then try some of the other visualizations explained above.

A common use case of tables is when you need to do a **cross-tabulation (crosstab)** of two categorical variables. Crosstabs, or *contingency tables*, count the number of elements of category A that also belong to category B, and present the results in a table:

| Type | Physical | Online |
| --- | --- | --- |
| Guitars | 40 | 60 |
| Acc. Pianos | 30 | 50 |

This table shows that there were 40 guitars sold in a physical location. To learn more about contingency tables, check [this](https://en.wikipedia.org/wiki/Contingency_table) out.

You can display a crosstab as-is in your dashboard if you like, but you can also slice it with a filter and then plot the relevant slice as a bar chart or a pie chart. Let's say you use a filter to slice out "online" (i.e. you only want to see sales from physical locations), which means the resulting plot can be:

<img src="data/img/physical_crosstab_plot.jpg" width="500"/>

In addition to these alternatives, you can also display single values using **scorecards**, like this:

<img src="data/img/scorecard.jpg" width="100"/>

## Applying our knowledge

Now that you know what kinds of visualizations go well with what kinds of data, it's time to choose which ones we'll use. Here's a table showing the variables that management requested to have in the dashboard, along with their data types:

| Variable | Type |
|-|-|
| Location | Categorical |
| State of delivery | Categorical |
| Time | Date |
| Customer age | Categorical (we'll use age group instead of simple ages) |
| Product type | Categorical |
| Product description | Categorical |
| Product line | Categorical |

These are the **dimensions** of our dashboards; i.e. the variables that we'll use to filter the data.

The numerical variables that our report will calculate and display are:

| Variable | Type |
|-|-|
| Revenue | Numeric (float) |
| Profit | Numeric (float) |
| Items sold | Numeric (integer) |
| Number of customers | Numeric (integer) |

These are called the **metrics** of our dashboard. You can say that metrics are numbers that are calculated and dimensions are filters that we use to determine which subsets of the data are used to compute those metrics.

<img src="data/img/metrics_and_dimensions.jpg" width="500"/>

The mathematical operations dashboards typically (but not exclusively) perform to create the metrics are:

1. **Sum**: sums all the values of the rows of that column
2. **Count**: counts how many rows there are in the selected column
3. **Count Distinct**: counts how many non-duplicate rows there are in the selected column
4. **Average**: finds the arithmetic mean of all the values of the rows of that column
5. **Minimum**: finds the minimum of all the values of the rows of that column
6. **Maximum**: finds the maximum of all the values of the rows of that column

### Exercise 1

What mathematical operations are most appropriate to:

1. Calculate the total number of items sold
2. Calculate the total number of customers
3. Calculate total profit
4. Calculate total revenue

**Answer**. Shown below:

1. **Count**. You would count the number of rows in the `transactions` table.
2. **Count Distinct**. You would count the number of unique customer identifiers in the `transactions` table. Since one single customer can make more than one purchase, you have to count distinct identifiers to avoid double-counting.
3. **Sum**. You would sum the profit earned from each transaction.
4. **Sum**. You would sum the revenue earned from each transaction in the `transactions` table.

### Exercise 2

This is a table with all the possible pairwise combinations of dimensions. For each combination, it is possible to compute one or more metrics.  For instance, with "Location" and "Customer age group" we can report how many items were sold by location and age group, or how much profit each age bracket produces in each location, or even how many customers from each age group buy in that location.

Please fill in this table with suitable visualization options (just filling the upper half is enough, as the bottom half is functionally equivalent).

| var/var | Location | State of delivery | Customer age group | Product type | Product description | Product line |
|-|-|-|-|-|-|-|
| Location | * | <> | <> | <> | <> | <> |
| State of delivery | <> | * | <> | <> | <> | <> |
| Customer age group | <> | <> | * | <> | <> | <> |
| Product type | <> | <> | <> | * | <> | <> |
| Product description | <> | <> | <> | <> | * | <> |
| Product line | <> | <> | <> | <> | <> | * |

**Answer**. One possible solution is shown below:

| var/var | Location | State of delivery | Customer age group | Product type | Product description | Product line |
|-|-|-|-|-|-|-|
| Location | * | Choropleth | Crosstab or bar/col/pie | Crosstab or bar/col/pie | Crosstab or bar/col/pie | Crosstab or bar/col/pie |
| State of delivery | <> | * | Choropleth | Choropleth | Choropleth | Choropleth |
| Customer age group | <> | <> | * | Crosstab or bar/col/pie | Crosstab or bar/col/pie | Crosstab or bar/col/pie |
| Product type | <> | <> | <> | * | Crosstab or bar/col/pie | Crosstab or bar/col/pie |
| Product description | <> | <> | <> | <> | * | Crosstab or bar/col/pie |
| Product line | <> | <> | <> | <> | <> | * |

The choice between column, bar, or pie chart depends a lot on aesthetics, ease of interpretation, and purpose. As we mentioned earlier, pie charts are good when you want to represent relative proportions of a few key categories. However, they are not recommended if you have many categories, or if you care about the absolute level of some quantity.

### Exercise 3

Let's say we want to plot revenue, profit, and sales over time. What would be an appropriate visualization?

**Answer.** Since we're dealing with dates, the best choice would be line plots. You can apply filters to the line plots so that the data only includes what you need. For instance, you could filter by a given age group (e.g. 50 - 60 years) and state of delivery (e.g. Michigan), and the plotted data would be updated accordingly. However, the filters wouldn't change the plot type itself.

## Planning out the layout of our dashboard

The table above was just a list of pairwise combinations of variables, but having a bunch of visualizations with no logical order whatsoever won't be pleasing to the eye, and most importantly won't convey any useful information.

When the end user opens the dashboard, they should see a clear, self-descriptive roadmap of what it has to offer. A good dashboard is like a restaurant menu that categorizes the meals, arranges them in hierarchies according to what the customer wants to read first, and provides only the information needed to make a decision. If the customer wants to learn more about a dish (e.g. the specific ingredients used or the name of the chef) they would have to ask the waiter or waitress. So, your dashboard should present the most relevant information first and show more detailed information only if the user asks for it.

Keep the following image in mind, it will help us as we move forward:

<img src="data/img/menu.jpg" width="500"/>

A good way to organize our dashboard is by creating **pages**. These are screens that show different aspects of your data, and the end user can switch between them by using a menu bar. Let's start with a basic set of pages - a "Bird's-Eye View" page (with an overview of the metrics) and a "Detailed View" page (which has more information and more filters).

Our dashboard will have three main layout components - a sidebar, a banner, and a body:

<img src="data/img/main_layout.jpg" width="500"/>

The sidebar will contain links to the pages. The banner will have the title of the page and optionally notes, date filters and scorecards. The page body will contain the visualizations.

### The "Bird's-Eye View" page layout

Recall the original business problem. The managers want to know:

1. How have our sales, revenue and profits evolved over time?
2. What are the locations with the highest numbers of sales and profit?
3. What are the items with the highest numbers of sales, profit and revenue across: product description, product type, product line, location, customer age group and state of delivery?

The third question has lots of dimensions, so we can put it in the "Detailed View" page. Let's put the first two questions in the "Bird's-Eye View" page.

We know that the first question can be answered using line plots. Since revenue and profits are both measured in dollars, we will show both of them in a single line plot using colored series, like this:

![A line plot](data/img/first_question_line_plot.png "A line plot")

Sales are measured in number of items, so they should have a line plot of their own. But what about the second question?

### Exercise 4

What visualization would be appropriate to answer the second question?

> 2. What are the locations with the highest numbers of sales and profit?

**Answer.** We can use bar plots. And again, since profit and revenue are measured in a common unit, they can be displayed in the same chart, grouped. Because the number of bars would be small (there are only four locations), this shoudn't be too difficult to interpret.

It would also be nice to give our users a summary of the whole page in a small set of aggregations. We can use scorecards for that. Let's create a scorecard for "Total revenue", one for "Total profit" and one for "Total sales", since these are the main metrics the managers are interested in. The layout of this page could be sketched like this:

<img src="data/img/birds_eye_layout.jpg" width="500"/>

As we said before, we are trying to make sure that the visualizations that are more important to the users appear first; that is, closer to the left-upper quadrant of the screen.

### The "Detailed View" page layout

Let's now move onto the third question. Since we want to know sales and total profit for each item, it makes sense to have a chart for sales (measured in number of items) and a chart for profit (measured in dollars). Items and dollars are both numeric (the former is an integer, the latter is a float). As items are categorical, bar charts would be fine. (Pie charts might not be easy to interpret because there are so many products and thus would get crowded.)

If you recall the `products` table, you'll remember that each product has a line ("Musical instruments"), a type ("Guitars"), and a description ("Fender American Stratocaster"). To avoid having to create a page-level filter for each one of these variables, we can add them to a single "Items" chart as **drill-down** variables. Drill-down allows you to select one element of a chart and move to a lower level in the hierarchy. The following GIF may be helpful to understand this concept than a mere description in words (this is a bar chart of sales by item, with drill-down by line, type, and description):

![Drill down](data/img/drill_down.gif "Drill down")

We can have a bar chart like this and next to it another bar chart (with no drill-down this time) that shows total profit for each *item*. We want our drill-down in the first chart to filter the second chart, so that when someone drills down by "Guitars", the second chart only shows guitars. This can be done easily in Tableau.

### Exercise 5

Now we have the two main charts of the "Detailed View" page, let's add some secondary charts (which will be smaller in size to represent their auxiliary status) for the dimensions that our management wants to filter the data by. These include location (number of items sold in each location), customer age group (number of customers in each age bracket) and state of delivery (number of items delivered). Which three plots would go well with these dimensions?

**Answer.** One possible answer is:

* Location - pie chart, since it would have only 4 slices (4 locations) and therefore be easy to interpret
* Customer age group - bar chart
* State of delivery - choropleth map (or geographic map with bubbles)

If we add some scorecards to display summaries, as well as a date picker, our layout could be like this:

<img src="data/img/detailed_view_layout.jpg" width="500"/>

**Note:** It is key that each visualization is linked to the question it is trying to answer. For each visualization, there should be a small text box stating the question and some other information that the user might find crucial to understand the chart.

Now we're all set. Let's move on to building our dashboard.

## Creating our dashboard in Tableau Desktop

### Loading our data

Let's open Tableau Desktop. The first thing we have to do is upload our data. Go to "Connect -> To a File -> Text File":

![Connect to CSV](data/img/t_connect_text_file.png "Connect to CSV File")

Select the `simply_music.csv` file and click "Upload":

![CSV upload](data/img/t_select_simplymusic.png "CSV upload")

After you have uploaded the table from your computer, you will see the "Data Source" page, which has two panels. The left one displays information about the data you just imported and the right one is the canvas, which allows you to create relationships between tables.

### Modeling our data

In our dataset, you will see the `simply_music.csv` file, including a preview of the columns and a few rows of data. A new tab will be created once the data is loaded. To start visualizing the data, click on "Go to Worksheet":

![Click on Worksheet](data/img/t_simplymusic_model.png "Click on Worksheet")

**Note:** This is where you would normally model your data - however, in our case we don't have any data that needs modeling because we are working with a single file that has been cleaned already, rather than multiple files from multiple sources.

At this point, save your report and name it "Simply Music 2018-2019 Report".

### Creating the visualizations

Tableau uses a workbook and sheet file structure, similar to Microsoft Excel (a single workbook contains one or more sheets). A sheet can be a worksheet, a dashboard, or a story. A **worksheet** shows the dataset and its metadata, a **dashboard** shows different views from multiple worksheets, and a **story** is a collection of worksheets or dashboards that together form a narrative from the data.

Before creating multiple worksheets, let's rename the worksheet from `Sheet 1` to `Location of Sales`, then add two dashboards and rename them from `Dashboard 1` to `Birds-Eye View`, and from `Dashboard 2` to `Detailed`:

![Rename Sheets and Add Dashboards](data/img/t_rename_worksheet_add_dashboard.png "Rename Sheets and Add Dashboards")

Remember that we have two pages - Bird's-Eye View and Detailed View. If you click on any of the worksheets or dashboards, they will be blank because we have not created any visualizations yet. Let's start populating the first dashboard "Birds-Eye View":

![Rename Page](data/img/t_dashboard_blank.png "Blank Dashboard")

## The Bird's-Eye View page

### Creating calculated fields

While on the "Location of Sales" worksheet, you'll see that there is no `PROFIT` column. Since this will be one of the main metrics of our dashboard, we need to create it. This is our definition of profit:

> PROFIT = DOLLARS_SOLD - TOTAL_COST

`TOTAL_COST` doesn't exist either, so we'll create it. The formula for this variable is:

> TOTAL_COST = UNITS_SOLD * COST

To create a calculated field, right-click in the whitespace of the data pane and click on "Create Calculated Field...". A box will appear with the name "Calculation1". Rename "Calculation1" to `TOTAL_COST` and write the following in the formula box:

```
[Units Sold] * [Cost]
```
![Total Cost](data/img/t_cost_calc.png "Total Cost")

Click "OK". We did the `TOTAL COST` calculation first because we need to reference the measure in the profit calculation. Now repeat the process, only this time name the new field `PROFIT` and put this in the formula box:

```
[Dollars Sold] - [Total Cost]
```
![Profit](data/img/t_profit_calc.png "Profit")

### Creating visualizations

When you build a view, you add fields from the Data pane. You can do this in [different ways](https://help.tableau.com/current/pro/desktop/en-us/buildmanual_dragging.htm) - for example by dragging data fields from the Data pane to the worksheet, double clicking them, selecting them and then clicking on "Show Me" to choose a chart type, or dropping them on the "Drop field here" grid, where you can then create the view.

### Exercise 6

Take a minute to find these options in the "Data" panel.

### Creating line charts

Now we have our `PROFIT` column and are ready to create the first chart. This will be a line chart according to the layout we planned. The dimension will be the date (since it is a time series plot) and the measures will be revenue and profit. Follow the steps below to build it:

1. Select both `PROFIT` and `Dollars Sold` by holding down "CTRL" on your keyboard, selecting both measures, and dragging them to the "Rows" shelf. Make sure that the measures are all set to "Sum" (remember Exercise 1) and change their names if needed (`Dollars Sold` -> `Revenue`).
2. Select the `Time Key` dimension and drag it to the "Columns" shelf.
3. Click the "Show Me" button and select "Lines (Dual)". If you select Lines (Continuous) you will need to create the Dual Axis by right-clicking on either `PROFIT` or `Dollars Sold` and selecting "Dual Axis".
4. Let's change the date granularity to monthly. Click the plus (+) symbol on the `Time Key` dimension twice until you see `MONTH`.

![Line Chart](data/img/t_line_chart.gif "Line Chart")

Once you finish creating your line chart, you can right click your Axis and click "Edit Axis" then update "Axis Titles" to "Month". Your chart should look something like this:

![Line Chart Final](data/img/t_line_chart_profit.png "Line Chart Final")

### Exercise 7

Carry out the above process for your `Sales` line chart.

**Answer.** Your chart should look like this:

![Line Chart Final](data/img/t_line_chart_sales.png "Line Chart Final")

### Creating bar charts

Now we have our `PROFIT` column and are ready to create the first chart. This will be a line chart, according to the layout we planned.

Now let's create the bar chart, according to the layout we planned. The dimensions will be Channel and the measures will be revenue and dollars sold. To build the bar chart follow the steps below:

* Select both `PROFIT` and `Dollars Sold` by holding down CTRL on your keyboard and selecting both measures and drag them to the "Rows" shelf. Make sure that the measures are all set to "Sum" (remember Exercise 1) and change their names if needed (`Dollars Sold` > `Revenue`)
* Select `Channel` dimension and drag them to the "Columns" shelf
* Click the "Show Me" button and select "Side by Side Bars". Next select "Swap Rows and Columns"

![Bar Chart](data/img/t_bar_chart.gif "Bar Chart")

### Exercise 8

Carry out the above process for your `Sales` bar chart.

**Answer.** Your chart should look like this:
![Bar Chart Final](data/img/t_bar_chart.png "Bar Chart Final")

### Creating the rest of the charts on the Bird's-Eye View page

Let's create the rest of our charts (including the scorecards) and add a new dashboard page. First we will add a visualization to the dashboard - in the "Sheets" pane, click on the sheet you would like to add and drag it onto the canvas. If you have setup containers, drag the visual within the container:

![Add Sheet](data/img/t_birdseye_linechart.gif "Add Sheet")

Second, we'll add the company logo (it is in the `data/img` folder as `logo.png`). To insert an image, go to the "Object" pane, then click on "Image" and drag it onto the dashboard canvas. Select the image by clicking "Choose", then find it and click "Fit Image -> OK":

![Add Logo](data/img/t_birdseye_logo.gif "Add Logo")

You'll notice the visualization colors don't align with the logo colors. Go to the sheet and adjust the colors to red and gray. To do so, go to the sheet and in the "Marks" section click on "Color" then select the red and gold palette. Click on `Dollars Sold` and select a shade of "Red". Select `PROFIT`, then select the grey palette and choose a shade, and finally select `PROFIT` again to assign the color:

![Change Colors](data/img/t_mark_color.gif "Change Colors")

Finally, we need to add a filter for our user to select a date range and update the charts in the dashboard accordingly. We are using a **data source** filter (which filters the data source directly), and we apply this filter by adding the `Time Key` to the "Filters" shelf. To do this, we need to go back to one of the sheets and drag the `Time Key` dimension to the "Filters" shelf. Then we will select "Range -> Next". We will not be changing the filter here, so leave the dates as the default (which is the earliest date in the dataset and the latest date in the dataset). Next, we right-click on the `Time Key` in the "Filters" shelf, then click "Apply to Worksheets -> All Using This Data Source". This applies the filter to any visualization from this dataset:

![Add Filters](data/img/t_filters.gif "Add Filters")

To learn more about filters, you can check [the documentation](https://help.tableau.com/current/pro/desktop/en-us/filtering.htm).

To add the filter to the dashboard, we return to the dashboard and select the visual. A down arrow will show up which leads to more available options. Select "Filters -> Time Key":

![Dashboard Filters](data/img/t_dashboard_filter.png "Dashboard Filter")

Now we can start editing the dashboard. To edit the name of a visual, double-click the title. To format the size or color of visuals, right click on the graphic and click "Format":

![Format](data/img/t_format.png "Format")

### Helpful design notes

When you start to master Tableau, you'll add containers so that you can control the design better. You [can](https://help.tableau.com/current/pro/desktop/en-us/dashboards_organize_floatingandtiled.htm) add horizontal or vertical containers to align your dashboard layout with the design we were provided earlier to organize the layout.

To align the content of your visuals to fit the full size of the container, go to the drop-down menu and click "Fit -> Entire View":

![Fit View](data/img/t_fitview.png "Fit View")

We add some more explanatory text, and our first page is done:

![Birds Eye View](data/img/t_birdseyeview.png "Birds Eye View")



### Exercise 9

Now do the same with the second page, the "Detailed View".

**Note 1:** We said we wanted to add drill-down to the dashboards. To achieve that, we must select each visual, click into its drop-down, and select "Use as Filter":

![How to drill down](data/img/t_visual_as_filter.png "How to drill down")

Once you apply this to all filters your dashboard will update dynamically.

**Note 2:** To add more context to a visual, drag a measure or dimension onto the "Tooltip" in the "Marks" section. This allows users to see more context when they hover over.

![Tooltip](data/img/t_tooltips.png "Tooltip")

**Note 3:** Make sure you update the `Number Format` of your measures to accurately reflect amounts that should be currency. To do this you need to go to a sheet and right-click on the measure, then go to "Default Properties -> Number Format":

![Number Format](data/img/t_number_format.png "Number Format")

To update multiple measures at once hold down "CTRL" while following the steps above:

![Currency Update](data/img/t_multiple_format.png "Currency Update")

**Note 4:** For the geographic map, we recommend you use a bubble map as the chart type but you will need to perform a cleaning step on the `Destination State` field to split the country and state fields so the map populates. Your map will be blank before you split the chart and when you click the indicator you will see these errors:

![Missing Locations](data/img/t_locations.png "Missing Locations")

Next, you will need to right-click the `Destination State` dimension and click "Split". Tableau will auto-detect how to split the field (there is a dash delimiter). You can now use your new field in your visual. (Ideally, you would do this before even loading your data into Tableau to prevent duplicative fields.)

![Splitting Dimensions](data/img/t_split_dimension.gif "Splitting Dimensions")

Now perform one final check of your map's locations, update accordingly if needed, and format colors to align to the Simply Music red and gray themes:

![Final Map](data/img/t_update_map.gif "Final Map")

**Answer**. Your second page should be similar to the below (we've tweaked the aesthetics just a bit, so don't worry if the look isn't exactly the same):

![Detailed](data/img/t_detailed.png "The end result")

## The end result

Here's a GIF of what our dashboard looks like now:

![Final Story](data/img/t_final_story.gif "Final Story")

Don't forget to publish your report to Tableau Public so you can share with anyone! You can download your report in PDF format as well ("File -> Print to PDF").

Your report is now ready to be presented to users. It will hopefully receive approval and be deployed using the live database.

## Conclusions

In this case, we looked at different types of visualizations and studied what kinds of data they are best suited to presenting. But we also saw that creating an analytical dashboard goes beyond this, and involves thinking about the concrete business questions that end users want to answer with the help of the dashboard. That is why we started by getting a grasp of the business process and the relevant dimensions and metrics, which informed the layout design and the choice of visualizations. Finally, we used Tableau to build and publish the dashboard using advanced options.

## Takeaways

Dashboard creation should always start with a sound understanding of the business process and the business questions. You also need to be familiar with the different types of visualizations and their strengths and weaknesses, and keep some basic design principles in mind as well. This allows you to build a dashboard that is functional but also not cluttered. Some key ideas you might want to keep in mind for the future are:

* *Dashboards are like restaurant menus.* They must be clear, straightforward and avoid overwhelming the user with too much information.
* *Your visualizations should be organized hierarchically.* The most important questions come first. Secondary questions can be placed more to the right or bottom, and be presented in a smaller size
* *Each visualization must answer a business question*, so it is always a good practice to label the chart or table with the question. You should also add explanatory text when you feel your users would appreciate it. However, if you find yourself adding too much text, then perhaps it is because your dashboard is not clear enough by itself and you should re-evaluate your layout or visualizations

Additionally, drill-down and charts that double as page-level filters can save you a lot of space because you don't need to add more controls (like drop-down lists) to act as filters in the page. Not everything should be drillable though, because the dashboard should be kept as simple as possible.

Tableau is an excellent dashboarding tool, and its graphical user interface makes it one of the easiest to use. We encourage you to explore the many possibilities that it offers. This will help you expand your data visualization vocabulary and hone your dashboarding skills at the same time.

## Attribution

"American restaurant menu", Eric Fischer, April 14th, 2018, [Creative Commons Attribution 2.0 generic license](https://creativecommons.org/licenses/by/2.0/deed.en), https://commons.wikimedia.org/wiki/File:American_restaurant_menu_(30085586928).jpg#file

